In [1]:
from muselsl import *
import time
import numpy as np
import matplotlib.pyplot as plt
from pylsl import StreamInlet, resolve_byprop
import json
import warnings
from command import Tracker, SHIFT_LENGTH, NUM_CHANNELS, EPOCH_LENGTH, OVERLAP_LENGTH, BUFFER_LENGTH, Band
import xml.dom.minidom

In [2]:
# -*- coding: utf-8 -*-
"""
Estimate Relaxation from Band Powers
This example shows how to buffer, epoch, and transform EEG data from a single
electrode into values for each of the classic frequencies (e.g. alpha, beta, theta)
Furthermore, it shows how ratios of the band powers can be used to estimate
mental state for neurofeedback.
The neurofeedback protocols described here are inspired by
*Neurofeedback: A Comprehensive Review on System Design, Methodology and Clinical Applications* by Marzbani et. al
Adapted from https://github.com/NeuroTechX/bci-workshop
"""

import numpy as np  # Module that simplifies computations on matrices
import matplotlib.pyplot as plt  # Module used for plotting
from pylsl import StreamInlet, resolve_byprop  # Module to receive EEG data
import utils  # Our own utility functions
from process_data import ChannelDataProcessor, Metrics
from PsychoPy_Code.BrainWave import run_psychopy

# Handy little enum to make code more readable



class Channel:
    TP0 = 0
    FP1 = 1
    FP2 = 2
    TP10 = 3
    DRL = 4



def _acquire_eeg_data_mock():
    """ Get eeg_data and timestamp from json for testing purposes. 

    :return: tuple: _eeg_data, _timestamp
    """
    json_file = open("to_send.json")
    k = json.loads(json_file.read())
    _eeg_data = k[0]["eeg_data"]
    _timestamp = k[0]["timestamp"]
    return _eeg_data, _timestamp

if __name__ == "__main__":

    """ 
    1. CONNECT TO EEG STREAM 
    
    """
     
    # Search for active LSL streams
    print('Looking for an EEG stream...')
    streams = resolve_byprop('type', 'EEG', timeout=2)
    if len(streams) == 0:
        raise RuntimeError('Can\'t find EEG stream.')
    
    # Set active EEG stream to inlet and apply time correction
    print("Start acquiring data")
    inlet = StreamInlet(streams[0], max_chunklen=12)
    eeg_time_correction = inlet.time_correction()
    
    # Get the stream info and description
    info = inlet.info()
    description = info.desc()
    
    """
    2. Obtain Threshold from PsycoPy 
    
    """
    
    cmd = Tracker(inlet=inlet, info=info)
    
    cmd.start_stage(mode='relax', stage=1)
    
    time.sleep()
    
    cmd.end_stage(mode='relax', stage=1)
    
    run_psychopy(cmd=cmd)
    cmd.update_stage_threshold()
    threshold = cmd.threshold

    """
    3. Record data and wave hand if metric is above threshold.  
    
    """
    
    # The try/except structure allows to quit the while loop by aborting the
    # script with <Ctrl-C>
    print('Press Ctrl-C in the console to break the while loop.')
    
    
    def get_fs(_info):
        return int(_info.nominal_srate())

    fs = get_fs(info)

    def _acquire_eeg_data(_inlet):
        """ Pull EEG data from inlet and return.

        :return: tuple: _eeg_data, _timestamp
        """
        _eeg_data, _timestamp = _inlet.pull_chunk(
            timeout=1, max_samples=int(SHIFT_LENGTH * fs))
        return _eeg_data, _timestamp


    try:
        # The following loop acquires data, computes band powers, and calculates neurofeedback metrics based on those band powers
        while True:
            #
            """ 3.1 ACQUIRE DATA """

            # Obtain EEG data from the LSL stream
            eeg_data, timestamp = _acquire_eeg_data(inlet)
            #             eeg_data, timestamp = _acquire_eeg_data_mock()

            c = ChannelDataProcessor(buffer_length=BUFFER_LENGTH,
                                     epoch_length=EPOCH_LENGTH,
                                     overlap_length=OVERLAP_LENGTH,
                                     shift_length=SHIFT_LENGTH, fs=fs, band_cls=Band)

            c.feed_new_data(eeg_data=eeg_data)  # Feed new data generated in the epoch
        
            metrics = []

            for i in range(NUM_CHANNELS):  # Iterate through all separate channels

                # Record channel smooth band power
                csbp = c.get_channel_smooth_band_powers(i) 
                # Run calculations on csbp to obtain desired metrics
                beta_metric = Metrics.beta_protocol(csbp, Band)
                # print("Beta metric: {}".format(beta_metric))
                metrics.append(beta_metric)
                
            metric = metrics[Channel.TP0]  # TODO: switch to the correct channel  
                
            def check_positive(_beta_metric, _threshold):
                """
                Returns True if beta_metric is less than the threshold 
                :return: 
                """
                return _beta_metric < _threshold
            
            def hand_wave():
                print("hand waved")
            
            def hand_stay():
                print("hand stayed still")
            
            if check_positive(_beta_metric=metric, _threshold=threshold): 
                hand_wave()
            else:
                hand_stay()
            
    except KeyboardInterrupt:
        print('Closing!')


#     dom = xml.dom.minidom.parseString(description.value())
#     print(info.channel_count)
#     print(description.value())
#     print(dom.toprettyxml())
# # description.value
    
    # # Get the sampling frequency
    # # This is an important value that represents how many EEG data points are
    # # collected in a second. This influences our frequency band calculation.
    # # for the Muse 2016, this should always be 256
    

    def get_fs_mock():
        warnings.warn("Using mock fs value: 256. Make sure that you are using get_fs outside of testing. ")
        return 256  # For testing purposes
    
#     fs = get_fs_mock() 

    """ 3. GET DATA """

    


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
Looking for an EEG stream...
Start acquiring data


ValueError: ctypes objects containing pointers cannot be pickled

In [14]:
thetas

[[0.027080094013804474,
  0.04307969962285625,
  0.04525204138904593,
  0.02820186549381042,
  -0.004988279819932072],
 [0.01289965989733113,
  0.0064893059633092534,
  0.002730737590208447,
  0.01620731016278457,
  -0.01993625672423425],
 [0.020874480186016904,
  0.03633918693416842,
  0.035144418062164035,
  0.022795347193007645,
  -0.0036355519059021687],
 [0.0046785261641279895,
  0.035137990301026364,
  0.039626674214974296,
  0.0052167566297191235,
  -0.001790045130212442],
 [-0.017099748633392108,
  0.040769724249240476,
  0.03792197236263317,
  -0.02239763931707026,
  -0.008966750689521633],
 [-0.005628688171333402,
  0.013453899684012031,
  0.021284359536505988,
  -0.011853563500576677,
  0.0010526149079206992],
 [-0.008758808611354755,
  0.010047064406250474,
  0.011814573018157347,
  -0.009297043175992867,
  -0.012185106389492571],
 [-0.014217121757514991,
  0.017327239100780805,
  0.023207703479525693,
  -0.021183657444975773,
  -0.014396035724004145],
 [-0.0023428152912038